## RAG의 컨텍스트 보강 검색
RAG(Retrieval-Augmented Generation)는 외부 소스에서 관련 지식을 검색하여 AI 응답의 품질을 향상시킵니다. 기존 검색 방식은 독립된 텍스트 조각(chunk)을 반환하기 때문에, 답변이 불완전할 수 있습니다.

이 문제를 해결하기 위해, 검색된 정보가 더 나은 일관성을 위해 주변 조각(chunk)을 포함하도록 보장하는 '컨텍스트 보강 검색(Context-Enriched Retrieval)'을 소개합니다.

> 💡 **초보자를 위한 설명:** 기존 RAG는 책에서 특정 문장 하나만 떼어와서 보여주는 것과 같습니다. 이 문장만으로는 전체 내용을 파악하기 어려울 수 있죠. '컨텍스트 보강 검색'은 그 문장의 앞뒤 문단까지 함께 가져와서 보여주는 것과 같습니다. 이렇게 하면 AI가 더 풍부한 맥락을 이해하고 더 정확하고 완전한 답변을 생성하는 데 도움이 됩니다.

### 이 노트북의 단계:
- **데이터 수집:** PDF에서 텍스트를 추출합니다.
- **컨텍스트 중첩 분할:** 컨텍스트를 보존하기 위해 텍스트를 중첩된 조각으로 분할합니다.
- **임베딩 생성:** 텍스트 조각을 숫자 표현(벡터)으로 변환합니다.
- **컨텍스트 인식 검색:** 더 나은 완전성을 위해 관련 조각을 주변 조각과 함께 검색합니다.
- **응답 생성:** 검색된 컨텍스트를 기반으로 언어 모델을 사용하여 응답을 생성합니다.
- **평가:** 모델 응답의 정확도를 평가합니다.

## 환경 설정
필요한 라이브러리를 가져오는 것으로 시작하겠습니다.

In [1]:
import fitz
import os
import numpy as np
import json
from openai import OpenAI

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 여기서는 PyMuPDF 라이브러리를 사용하여 PDF 파일에서 텍스트를 추출합니다.

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
    pdf_path (str): PDF 파일 경로.

    Returns:
    str: PDF에서 추출된 텍스트.
    """
    # PDF 파일을 엽니다
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 빈 문자열을 초기화합니다

    # PDF의 각 페이지를 반복합니다
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 페이지를 가져옵니다
        text = page.get_text("text")  # 페이지에서 텍스트를 추출합니다
        all_text += text  # 추출된 텍스트를 all_text 문자열에 추가합니다

    return all_text  # 추출된 텍스트를 반환합니다

## 추출된 텍스트 분할하기
추출된 텍스트가 준비되면, 검색 정확도를 높이기 위해 더 작고 중첩되는 조각(chunk)으로 나눕니다.

> 💡 **초보자를 위한 설명:** 텍스트를 조각으로 나눌 때, 의미가 이어지는 부분이 잘릴 수 있습니다. 예를 들어, 'A는 B이다. 그리고 B는 C이다.'라는 문장이 있을 때, 'A는 B이다.'와 '그리고 B는 C이다.'로 나누면 각 조각의 의미가 불완전해집니다. '중첩(Overlapping)'은 각 조각의 끝부분과 다음 조각의 시작 부분을 겹치게 만들어 이러한 정보 손실을 줄이는 기법입니다. 이렇게 하면 문맥이 유지되어 검색 품질이 향상됩니다.

In [3]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 중첩을 포함하여 n개의 문자 세그먼트로 분할합니다.

    Args:
    text (str): 분할할 텍스트.
    n (int): 각 조각의 문자 수.
    overlap (int): 조각 간에 겹치는 문자 수.

    Returns:
    List[str]: 텍스트 조각 리스트.
    """
    chunks = []  # 조각을 저장할 빈 리스트를 초기화합니다
    
    # (n - overlap) 크기의 스텝으로 텍스트를 반복합니다
    for i in range(0, len(text), n - overlap):
        # 인덱스 i부터 i + n까지의 텍스트 조각을 chunks 리스트에 추가합니다
        chunks.append(text[i:i + n])

    return chunks  # 텍스트 조각 리스트를 반환합니다

## OpenAI API 클라이언트 설정
임베딩과 응답을 생성하기 위해 OpenAI 클라이언트를 초기화합니다.

In [ ]:
# 기본 URL과 API 키로 OpenAI 클라이언트를 초기화합니다
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키를 가져옵니다
)

## PDF 파일에서 텍스트 추출 및 분할
이제 PDF를 로드하고, 텍스트를 추출한 다음, 조각으로 분할합니다.

In [5]:
# PDF 파일 경로를 정의합니다
pdf_path = "data/AI_Information.pdf"

# PDF 파일에서 텍스트를 추출합니다
extracted_text = extract_text_from_pdf(pdf_path)

# 추출된 텍스트를 1000자 단위로 분할하되, 200자의 중첩을 둡니다
text_chunks = chunk_text(extracted_text, 1000, 200)

# 생성된 텍스트 조각의 수를 출력합니다
print("Number of text chunks:", len(text_chunks))

# 첫 번째 텍스트 조각을 출력합니다
print("\nFirst text chunk:")
print(text_chunks[0])

Number of text chunks: 42

First text chunk:
Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving 
and 

## 텍스트 조각에 대한 임베딩 생성
임베딩은 텍스트를 숫자 벡터로 변환하여 효율적인 유사도 검색을 가능하게 합니다.

> 💡 **초보자를 위한 설명:** 임베딩은 '단어를 숫자로 된 좌표에 표시하는 것'과 같습니다. 컴퓨터는 글자를 직접 이해하지 못하므로, 의미가 비슷한 단어나 문장을 가까운 위치의 숫자로 바꿔줍니다. 이렇게 하면 '인공지능'과 '머신러닝'처럼 의미가 비슷한 단어들이 벡터 공간에서 가까운 거리에 위치하게 되어, 컴퓨터가 의미 기반의 유사도 검색을 효율적으로 수행할 수 있게 됩니다.

In [6]:
def create_embeddings(text, model="BAAI/bge-en-icl"):
    """
    지정된 OpenAI 모델을 사용하여 주어진 텍스트에 대한 임베딩을 생성합니다.

    Args:
    text (str): 임베딩을 생성할 입력 텍스트.
    model (str): 임베딩 생성에 사용할 모델. 기본값은 "BAAI/bge-en-icl"입니다.

    Returns:
    dict: 임베딩을 포함하는 OpenAI API의 응답.
    """
    # 지정된 모델을 사용하여 입력 텍스트에 대한 임베딩을 생성합니다
    response = client.embeddings.create(
        model=model,
        input=text
    )

    return response  # 임베딩이 포함된 응답을 반환합니다

# 텍스트 조각에 대한 임베딩을 생성합니다
response = create_embeddings(text_chunks)

## 컨텍스트 인식 의미론적 검색 구현
더 나은 컨텍스트를 위해 주변 조각을 포함하도록 검색을 수정합니다.

> 💡 **초보자를 위한 설명:** 이것이 바로 '컨텍스트 보강 검색'의 핵심입니다. 사용자의 질문과 가장 관련성이 높은 텍스트 조각을 찾은 뒤, 그 조각만 떼어오는 것이 아니라 그 앞뒤에 있는 조각까지 함께 가져옵니다. 이렇게 하면 AI가 질문에 대한 답변을 생성할 때 더 넓은 문맥을 참고할 수 있어, 훨씬 자연스럽고 정확한 답변을 할 수 있게 됩니다.

In [7]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터.
    vec2 (np.ndarray): 두 번째 벡터.

    Returns:
    float: 두 벡터 간의 코사인 유사도.
    """
    # 두 벡터의 내적을 계산하고 각 벡터의 노름(norm)의 곱으로 나눕니다
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [8]:
def context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1):
    """
    가장 관련성 높은 조각을 주변 조각과 함께 검색합니다.

    Args:
    query (str): 검색 질문.
    text_chunks (List[str]): 텍스트 조각 리스트.
    embeddings (List[dict]): 조각 임베딩 리스트.
    k (int): 검색할 관련 조각의 수.
    context_size (int): 포함할 주변 조각의 수.

    Returns:
    List[str]: 컨텍스트 정보가 포함된 관련 텍스트 조각.
    """
    # 질문을 임베딩 벡터로 변환합니다
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []

    # 질문과 각 텍스트 조각 임베딩 간의 유사도 점수를 계산합니다
    for i, chunk_embedding in enumerate(embeddings):
        # 질문 임베딩과 현재 조각 임베딩 간의 코사인 유사도를 계산합니다
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding.embedding))
        # 인덱스와 유사도 점수를 튜플로 저장합니다
        similarity_scores.append((i, similarity_score))

    # 유사도 점수를 기준으로 조각을 내림차순으로 정렬합니다 (유사도가 가장 높은 것이 먼저 오도록)
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # 가장 관련성 높은 조각의 인덱스를 가져옵니다
    top_index = similarity_scores[0][0]

    # 컨텍스트 포함 범위를 정의합니다
    # 0 미만이 되거나 text_chunks의 길이를 초과하지 않도록 보장합니다
    start = max(0, top_index - context_size)
    end = min(len(text_chunks), top_index + context_size + 1)

    # 관련 조각을 주변 컨텍스트 조각과 함께 반환합니다
    return [text_chunks[i] for i in range(start, end)]

## 컨텍스트 검색을 사용하여 질문 실행하기
이제 컨텍스트 보강 검색을 테스트합니다.

In [9]:
# JSON 파일에서 검증 데이터셋을 로드합니다
with open('data/val.json') as f:
    data = json.load(f)

# 데이터셋에서 첫 번째 질문을 추출하여 쿼리로 사용합니다
query = data[0]['question']

# 컨텍스트를 위해 가장 관련성 높은 조각과 그 주변 조각을 검색합니다
# 매개변수:
# - query: 검색할 질문
# - text_chunks: PDF에서 추출한 텍스트 조각
# - response.data: 텍스트 조각의 임베딩
# - k=1: 가장 일치하는 항목 1개를 반환
# - context_size=1: 컨텍스트를 위해 최상위 일치 항목의 앞뒤로 1개의 조각을 포함
top_chunks = context_enriched_search(query, text_chunks, response.data, k=1, context_size=1)

# 참고용으로 질문을 출력합니다
print("Query:", query)
# 검색된 각 조각을 제목과 구분 기호와 함께 출력합니다
for i, chunk in enumerate(top_chunks):
    print(f"Context {i + 1}:\n{chunk}\n=====================================")

Query: What is 'Explainable AI' and why is it considered important?
Context 1:
nt aligns with societal values. Education and awareness campaigns inform the public 
about AI, its impacts, and its potential. 
Chapter 19: AI and Ethics 
Principles of Ethical AI 
Ethical AI principles guide the development and deployment of AI systems to ensure they are fair, 
transparent, accountable, and beneficial to society. Key principles include respect for human 
rights, privacy, non-discrimination, and beneficence. 
 
 
Addressing Bias in AI 
AI systems can inherit and amplify biases present in the data they are trained on, leading to unfair 
or discriminatory outcomes. Addressing bias requires careful data collection, algorithm design, 
and ongoing monitoring and evaluation. 
Transparency and Explainability 
Transparency and explainability are essential for building trust in AI systems. Explainable AI (XAI) 
techniques aim to make AI decisions more understandable, enabling users to assess their 
f

## 검색된 컨텍스트를 사용하여 응답 생성하기
이제 LLM을 사용하여 응답을 생성합니다.

In [10]:
# AI 어시스턴트에 대한 시스템 프롬프트를 정의합니다
system_prompt = "당신은 주어진 컨텍스트를 기반으로만 엄격하게 답변하는 AI 어시스턴트입니다. 제공된 컨텍스트에서 직접 답변을 도출할 수 없는 경우, '답변하기에 충분한 정보가 없습니다.'라고 응답하세요."

def generate_response(system_prompt, user_message, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델로부터 응답을 생성합니다.

    Args:
    system_prompt (str): AI의 행동을 안내하는 시스템 프롬프트.
    user_message (str): 사용자의 메시지 또는 질문.
    model (str): 응답 생성에 사용할 모델. 기본값은 "meta-llama/Llama-2-7B-chat-hf"입니다.

    Returns:
    dict: AI 모델의 응답.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 최상위 조각을 기반으로 사용자 프롬프트를 생성합니다
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# AI 응답을 생성합니다
ai_response = generate_response(system_prompt, user_prompt)

## AI 응답 평가하기
AI의 응답을 기대 답변과 비교하여 점수를 매깁니다.

In [11]:
# 평가 시스템에 대한 시스템 프롬프트를 정의합니다
evaluate_system_prompt = "당신은 AI 어시스턴트의 응답을 평가하는 지능형 평가 시스템입니다. AI 어시스턴트의 응답이 실제 응답과 매우 가까우면 1점을 부여하세요. 응답이 실제 응답과 비교하여 부정확하거나 만족스럽지 않으면 0점을 부여하세요. 응답이 실제 응답과 부분적으로 일치하면 0.5점을 부여하세요."

# 사용자 질문, AI 응답, 실제 응답 및 평가 시스템 프롬프트를 결합하여 평가 프롬프트를 생성합니다
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# 평가 시스템 프롬프트와 평가 프롬프트를 사용하여 평가 응답을 생성합니다
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 평가 응답을 출력합니다
print(evaluation_response.choices[0].message.content)

Based on the evaluation criteria, I would assign a score of 0.8 to the AI assistant's response.

The response is very close to the true response, and it correctly conveys the main idea of Explainable AI (XAI) and its importance. The AI assistant's response is also well-structured and easy to understand, which is a positive aspect.

However, there are a few minor differences between the AI assistant's response and the true response. The AI assistant's response is slightly more detailed and provides additional points (1-4) that are not present in the true response. Additionally, the AI assistant's response uses more formal language and phrases, such as "In essence," which is not present in the true response.

Despite these minor differences, the AI assistant's response is still very close to the true response, and it effectively conveys the main idea of XAI and its importance. Therefore, I would assign a score of 0.8.
